## Import libaries

In [383]:
import pandas as pd
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import explained_variance_score, mean_squared_error, mean_absolute_error
import sklearn.metrics as metrics
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor, GradientBoostingRegressor
import xgboost as xgb
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import numpy as np

## Completely random faux dataset

In [331]:
demand = []
for i in range(0,100):
    n = random.randint(0,10000)
    demand.append(n)

product_price = []

for i in range(0,100):
    n = random.randint(10,1000)
    product_price.append(n)

date = []
for i in range(0,100):
    date.append("t - " + str(i))

skews_list = ['3-iron','4-iron','5-iron','6-iron','7-iron','8-iron','9-iron','driver','1-wood','2-wood','3-wood','4-wood',
        '5-wood','6-wood','7-wood','3-hybrid','4-hybrid','5-hybrid','pitching wedge','approach wedge','sand wedge',
        'lob wedge', 'putter']
skews = []
for i in range(0,100):
    n = random.choice(skews_list)
    skews.append(n)

In [332]:
# Define a dictionary containing Students data
data = {'date': date,
        'demand': demand,
        'product_price': product_price,
        'skews': skews,
       }
  
# Convert the dictionary into DataFrame
dataset = pd.DataFrame(data)
dataset.head()

,date,demand,product_price,skews
0,t - 0,9103,356,5-wood
1,t - 1,2684,791,driver
2,t - 2,5025,195,1-wood
3,t - 3,2389,469,6-wood
4,t - 4,5649,373,7-wood


## One-Hot Encode

#### Separate the two datasets here into a cross-sectional one, and a longitudinal as the approaches diverge here

In [333]:
dataset_long = dataset
dataset_cross = dataset.loc[:, ['demand','skews','product_price']]

In [334]:
dataset_cross = pd.get_dummies(dataset_cross, prefix=['skews'])

In [335]:
pd.set_option('display.max_columns', None)
dataset_cross.head()

,demand,product_price,skews_1-wood,skews_2-wood,skews_3-hybrid,skews_3-iron,skews_3-wood,skews_4-hybrid,skews_4-iron,skews_4-wood,skews_5-hybrid,skews_5-iron,skews_5-wood,skews_6-iron,skews_6-wood,skews_7-iron,skews_7-wood,skews_8-iron,skews_9-iron,skews_approach wedge,skews_driver,skews_lob wedge,skews_pitching wedge,skews_putter,skews_sand wedge
0,9103,356,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,2684,791,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,5025,195,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2389,469,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5649,373,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0


#### We could filter the dataset here based on skew

In [336]:
## filter the data via skew

In [337]:
len(dataset_cross.columns)

25

## Split the dataset into feature/target label 

In [338]:
x = dataset_cross.loc[:, ["demand","skews_1-wood","skews_2-wood","skews_3-hybrid","skews_3-iron",
                   "skews_3-wood","skews_4-hybrid","skews_4-iron","skews_4-wood","skews_5-hybrid","skews_5-iron",
                   "skews_5-wood","skews_6-iron","skews_6-wood","skews_7-iron","skews_7-wood","skews_8-iron",
                   "skews_9-iron","skews_approach wedge","skews_driver","skews_lob wedge","skews_pitching wedge",
                   "skews_putter", "skews_sand wedge"]]
y = dataset_cross.loc[:, ["product_price"]]

In [339]:
print(len(x.columns))
print(len(y.columns))

24
1


## Split the dataset into training and test sets

In [340]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=4)

In [341]:
x_train.head()

,demand,skews_1-wood,skews_2-wood,skews_3-hybrid,skews_3-iron,skews_3-wood,skews_4-hybrid,skews_4-iron,skews_4-wood,skews_5-hybrid,skews_5-iron,skews_5-wood,skews_6-iron,skews_6-wood,skews_7-iron,skews_7-wood,skews_8-iron,skews_9-iron,skews_approach wedge,skews_driver,skews_lob wedge,skews_pitching wedge,skews_putter,skews_sand wedge
80,945,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,5649,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
81,4079,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
76,698,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
85,5470,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [342]:
y_train.head()

,product_price
80,555
4,373
81,920
76,809
85,547


In [343]:
x_test.head()

,demand,skews_1-wood,skews_2-wood,skews_3-hybrid,skews_3-iron,skews_3-wood,skews_4-hybrid,skews_4-iron,skews_4-wood,skews_5-hybrid,skews_5-iron,skews_5-wood,skews_6-iron,skews_6-wood,skews_7-iron,skews_7-wood,skews_8-iron,skews_9-iron,skews_approach wedge,skews_driver,skews_lob wedge,skews_pitching wedge,skews_putter,skews_sand wedge
20,7053,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
10,6717,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
96,6902,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16,4421,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
63,2757,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [344]:
y_test.head()

,product_price
20,514
10,322
96,264
16,168
63,254


## Scale the features so no bias based on range of values

In [345]:
scaler = MinMaxScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

In [346]:
print(x_train_scaled)

[[0.09043091 0.         0.         ... 0.         0.         0.        ]
 [0.56625531 0.         0.         ... 0.         0.         0.        ]
 [0.40744487 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.70008092 0.         0.         ... 0.         0.         0.        ]
 [0.23295569 0.         0.         ... 0.         0.         0.        ]
 [0.04248432 0.         0.         ... 0.         0.         0.        ]]


In [347]:
print(y_train)

    product_price
80            555
4             373
81            920
76            809
85            547
..            ...
87            685
1             791
69            509
55            281
46            531

[80 rows x 1 columns]


In [348]:
print(x_test_scaled)

[[0.70827433 0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         1.         0.         0.         0.        ]
 [0.67428687 0.         0.         0.         1.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.6930002  0.         1.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.44203925 0.         0.         0.         0.         1.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.

In [349]:
print(y_test)

    product_price
20            514
10            322
96            264
16            168
63            254
24            263
53            274
97            919
41             55
47            189
43            676
2             195
95            929
26            499
13            853
37            354
14            321
29            219
35            326
54            673


## Modeling

### Cross-Sectional Models (likely sub-optimal approach compared to longitudinal, or filtered longitudinal, but can try them)
*Note, all models are using default hyperparameters for now, which is severely sub-optimal

#### Linear Regression

In [350]:
lin_reg = LinearRegression()
model_1 = lin_reg.fit(x_train_scaled,y_train)
y_predict = model_1.predict(x_test)
lin_reg_test_mse = metrics.mean_squared_error(y_test, y_predict)
lin_reg_test_mae = metrics.mean_absolute_error(y_test, y_predict)
lin_reg_test_evs = explained_variance_score(y_test, y_predict)
print("This is the test mean-squared error (the lower the value the better): ", lin_reg_test_mse)
print("This is the test mean-absolute error (the lower the value the better): ", lin_reg_test_mae)
print("This is the test explained variance score (best possible score is 1.0, lower values are worse): ", lin_reg_test_evs)

This is the test mean-squared error (the lower the value the better):  1057219908831.5725
This is the test mean-absolute error (the lower the value the better):  965151.9690645523
This is the test explained variance score (best possible score is 1.0, lower values are worse):  -1917820.9895716517


/Users/tylerkapp/Library/Python/3.10/lib/python/site-packages/scikit_learn-1.2.0-py3.10-macosx-12-arm64.egg/sklearn/base.py:402: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


#### 2nd Degree Polynomial Regression

In [351]:
poly_reg_2 = make_pipeline(PolynomialFeatures(2),LinearRegression())
model_2 = poly_reg_2.fit(x_train_scaled,y_train)
y_predict = model_2.predict(x_test)
poly_reg_2_test_mse = metrics.mean_squared_error(y_test, y_predict)
poly_reg_2_test_mae = metrics.mean_absolute_error(y_test, y_predict)
poly_reg_2_test_evs = explained_variance_score(y_test, y_predict)
print("This is the test mean-squared error (the lower the value the better): ", poly_reg_2_test_mse)
print("This is the test mean-absolute error (the lower the value the better): ", poly_reg_2_test_mae)
print("This is the test explained variance score (best possible score is 1.0, lower values are worse): ", poly_reg_2_test_evs)

This is the test mean-squared error (the lower the value the better):  5.8066564661562504e+38
This is the test mean-absolute error (the lower the value the better):  7.375898919303588e+18
This is the test explained variance score (best possible score is 1.0, lower values are worse):  -8.029145991030782e+33


/Users/tylerkapp/Library/Python/3.10/lib/python/site-packages/scikit_learn-1.2.0-py3.10-macosx-12-arm64.egg/sklearn/base.py:402: UserWarning: X has feature names, but PolynomialFeatures was fitted without feature names
  warnings.warn(


#### 3rd Degree Polynomial Regression

In [352]:
poly_reg_3 =make_pipeline(PolynomialFeatures(3),LinearRegression())
model_3 = poly_reg_3.fit(x_train_scaled,y_train)
y_predict = model_3.predict(x_test)
poly_reg_3_test_mse = metrics.mean_squared_error(y_test, y_predict)
poly_reg_3_test_mae = metrics.mean_absolute_error(y_test, y_predict)
poly_reg_3_test_evs = explained_variance_score(y_test, y_predict)
print("This is the test mean-squared error (the lower the value the better): ", poly_reg_3_test_mse)
print("This is the test mean-absolute error (the lower the value the better): ", poly_reg_3_test_mae)
print("This is the test explained variance score (best possible score is 1.0, lower values are worse): ", poly_reg_3_test_evs)

This is the test mean-squared error (the lower the value the better):  5.637497229631959e+45
This is the test mean-absolute error (the lower the value the better):  2.793227017504088e+22
This is the test explained variance score (best possible score is 1.0, lower values are worse):  -7.410733072870072e+40


/Users/tylerkapp/Library/Python/3.10/lib/python/site-packages/scikit_learn-1.2.0-py3.10-macosx-12-arm64.egg/sklearn/base.py:402: UserWarning: X has feature names, but PolynomialFeatures was fitted without feature names
  warnings.warn(


#### Support Vector Machine Regression

In [353]:
svr_reg = SVR()
model_4 = svr_reg.fit(x_train_scaled,y_train)
y_predict = model_4.predict(x_test)
svr_test_mse = metrics.mean_squared_error(y_test, y_predict)
svr_test_mae = metrics.mean_absolute_error(y_test, y_predict)
svr_test_evs = explained_variance_score(y_test, y_predict)
print("This is the test mean-squared error (the lower the value the better): ", svr_test_mse)
print("This is the test mean-absolute error (the lower the value the better): ", svr_test_mae)
print("This is the test explained variance score (best possible score is 1.0, lower values are worse): ", svr_test_evs)

This is the test mean-squared error (the lower the value the better):  81812.07368658832
This is the test mean-absolute error (the lower the value the better):  262.09832159020004
This is the test explained variance score (best possible score is 1.0, lower values are worse):  2.220446049250313e-16


/Users/tylerkapp/Library/Python/3.10/lib/python/site-packages/scikit_learn-1.2.0-py3.10-macosx-12-arm64.egg/sklearn/utils/validation.py:1141: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/tylerkapp/Library/Python/3.10/lib/python/site-packages/scikit_learn-1.2.0-py3.10-macosx-12-arm64.egg/sklearn/base.py:402: UserWarning: X has feature names, but SVR was fitted without feature names
  warnings.warn(


#### Decision Tree Regression

In [354]:
dt_reg = DecisionTreeRegressor()
model_5 = dt_reg.fit(x_train_scaled,y_train)
y_predict = model_5.predict(x_test)
dt_test_mse = metrics.mean_squared_error(y_test, y_predict)
dt_test_mae = metrics.mean_absolute_error(y_test, y_predict)
dt_test_evs = explained_variance_score(y_test, y_predict)
print("This is the test mean-squared error (the lower the value the better): ", dt_test_mse)
print("This is the test mean-absolute error (the lower the value the better): ", dt_test_mae)
print("This is the test explained variance score (best possible score is 1.0, lower values are worse): ", dt_test_evs)

This is the test mean-squared error (the lower the value the better):  374290.85
This is the test mean-absolute error (the lower the value the better):  555.65
This is the test explained variance score (best possible score is 1.0, lower values are worse):  0.0


/Users/tylerkapp/Library/Python/3.10/lib/python/site-packages/scikit_learn-1.2.0-py3.10-macosx-12-arm64.egg/sklearn/base.py:402: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(


#### Random Forest Regression

In [355]:
rf_reg = RandomForestRegressor()
model_6 = rf_reg.fit(x_train_scaled,y_train)
y_predict = model_6.predict(x_test)
rf_test_mse = metrics.mean_squared_error(y_test, y_predict)
rf_test_mae = metrics.mean_absolute_error(y_test, y_predict)
rf_test_evs = explained_variance_score(y_test, y_predict)
print("This is the test mean-squared error (the lower the value the better): ", rf_test_mse)
print("This is the test mean-absolute error (the lower the value the better): ", rf_test_mae)
print("This is the test explained variance score (best possible score is 1.0, lower values are worse): ", rf_test_evs)

This is the test mean-squared error (the lower the value the better):  188345.81537499998
This is the test mean-absolute error (the lower the value the better):  392.9395
This is the test explained variance score (best possible score is 1.0, lower values are worse):  -0.09055511006339967


/var/folders/t5/frwy1jfs7b31fsm3w2ln0vp80000gn/T/ipykernel_2343/1623376634.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model_6 = rf_reg.fit(x_train_scaled,y_train)
/Users/tylerkapp/Library/Python/3.10/lib/python/site-packages/scikit_learn-1.2.0-py3.10-macosx-12-arm64.egg/sklearn/base.py:402: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


#### Bagging Regressor

In [356]:
br_reg = BaggingRegressor()
model_7 = br_reg.fit(x_train_scaled,y_train)
y_predict = model_7.predict(x_test)
bg_test_mse = metrics.mean_squared_error(y_test, y_predict)
bg_test_mae = metrics.mean_absolute_error(y_test, y_predict)
bg_test_evs = explained_variance_score(y_test, y_predict)
print("This is the test mean-squared error (the lower the value the better): ", bg_test_mse)
print("This is the test mean-absolute error (the lower the value the better): ", bg_test_mae)
print("This is the test explained variance score (best possible score is 1.0, lower values are worse): ", bg_test_evs)

This is the test mean-squared error (the lower the value the better):  145765.027
This is the test mean-absolute error (the lower the value the better):  334.08000000000004
This is the test explained variance score (best possible score is 1.0, lower values are worse):  -0.24873619451626539


/Users/tylerkapp/Library/Python/3.10/lib/python/site-packages/scikit_learn-1.2.0-py3.10-macosx-12-arm64.egg/sklearn/ensemble/_bagging.py:510: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)
/Users/tylerkapp/Library/Python/3.10/lib/python/site-packages/scikit_learn-1.2.0-py3.10-macosx-12-arm64.egg/sklearn/base.py:402: UserWarning: X has feature names, but BaggingRegressor was fitted without feature names
  warnings.warn(


#### Gradient Boosting Regressor

In [357]:
grad_boost_reg = BaggingRegressor()
model_8 = grad_boost_reg.fit(x_train_scaled,y_train)
y_predict = model_8.predict(x_test)
grad_boost_test_mse = metrics.mean_squared_error(y_test, y_predict)
grad_boost_test_mae = metrics.mean_absolute_error(y_test, y_predict)
grad_boost_test_evs = explained_variance_score(y_test, y_predict)
print("This is the test mean-squared error (the lower the value the better): ", grad_boost_test_mse)
print("This is the test mean-absolute error (the lower the value the better): ", grad_boost_test_mae)
print("This is the test explained variance score (best possible score is 1.0, lower values are worse): ", grad_boost_test_evs)

This is the test mean-squared error (the lower the value the better):  179060.25449999998
This is the test mean-absolute error (the lower the value the better):  384.415
This is the test explained variance score (best possible score is 1.0, lower values are worse):  -0.4248442996492694


/Users/tylerkapp/Library/Python/3.10/lib/python/site-packages/scikit_learn-1.2.0-py3.10-macosx-12-arm64.egg/sklearn/ensemble/_bagging.py:510: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return column_or_1d(y, warn=True)
/Users/tylerkapp/Library/Python/3.10/lib/python/site-packages/scikit_learn-1.2.0-py3.10-macosx-12-arm64.egg/sklearn/base.py:402: UserWarning: X has feature names, but BaggingRegressor was fitted without feature names
  warnings.warn(


#### XGBoost Regressor

In [358]:
xgb_boost_reg = xgb.XGBRegressor()
model_9 = xgb_boost_reg.fit(x_train_scaled,y_train)
y_predict = model_9.predict(x_test)
xgb_test_mse = metrics.mean_squared_error(y_test, y_predict)
xgb_test_mae = metrics.mean_absolute_error(y_test, y_predict)
xgb_test_evs = explained_variance_score(y_test, y_predict)
print("This is the test mean-squared error (the lower the value the better): ", xgb_test_mse)
print("This is the test mean-absolute error (the lower the value the better): ", xgb_test_mae)
print("This is the test explained variance score (best possible score is 1.0, lower values are worse): ", xgb_test_evs)

This is the test mean-squared error (the lower the value the better):  257976.9552059412
This is the test mean-absolute error (the lower the value the better):  454.290771484375
This is the test explained variance score (best possible score is 1.0, lower values are worse):  -0.10822837098213833


### Longitudinal Models (better approach than cross-sectional)
**Note, many model architectures have not been attempted yet, and this is a pretty simple model that has not been tuned, on random data, but nonetheless is assuredly better than what they're doing now.

In [359]:
dataset_long.head()

,date,demand,product_price,skews
0,t - 0,9103,356,5-wood
1,t - 1,2684,791,driver
2,t - 2,5025,195,1-wood
3,t - 3,2389,469,6-wood
4,t - 4,5649,373,7-wood


In [360]:
# could filter the dataset here based on skews

## Split Into train/test

In [387]:
train = dataset_long.loc[1:, ['date', 'product_price']]
test = dataset_long.loc[0, dataset_long.columns == 'product_price']

In [388]:
train.head()

,date,product_price
1,t - 1,791
2,t - 2,195
3,t - 3,469
4,t - 4,373
5,t - 5,312


In [389]:
test.head() # true test label

product_price    356
Name: 0, dtype: object

In [391]:
# remove the date for train object
train = train.loc[:, train.columns == 'product_price']
train.head()

,product_price
1,791
2,195
3,469
4,373
5,312


In [393]:
# convert to train & test sets numpy array
train = np.asarray(train)
test = np.asarray(test)

#### Holt-Winters Exponential Smoothing

In [400]:
train = np.asarray(train)
model = ExponentialSmoothing(train) 

results = model.fit() 
forecast = results.forecast() 
predicted = forecast[0]
print('Holt-Winters Exponential Smoothing model predicttion of price at time t-0 (current time) = ', predicted)
print('Actual price at time t-0 (current time) = ', int(test))

Holt-Winters Exponential Smoothing model predicttion of price at time t-0 (current time) =  491.6195054444226
Actual price at time t-0 (current time) =  356
